In [106]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Input, TimeDistributed, LeakyReLU, BatchNormalization
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau,TensorBoard

from sklearn.metrics import mean_squared_error
from math import sqrt

In [2]:
!unzip data.zip

Archive:  data.zip
   creating: Train/
  inflating: Train/bangkok_Train.csv  
  inflating: Train/chiangmai_Train.csv  
  inflating: Train/chanthaburi_Train.csv  
  inflating: Train/songkhla_Train.csv  
  inflating: Train/khonkaen_Train.csv  
  inflating: Train/kanchanaburi_Train.csv  
   creating: Test/
  inflating: Test/kanchanaburi_Test.csv  
  inflating: Test/chiangmai_Test.csv  
  inflating: Test/chanthaburi_Test.csv  
  inflating: Test/khonkaen_Test.csv  
  inflating: Test/bangkok_Test.csv   
  inflating: Test/songkhla_Test.csv  


# Feature Engineering

In [28]:
fmt_prv = ['chanthaburi','chiangmai','kanchanaburi','bangkok','khonkaen','songkhla']

########################################
#                                      #
#         Feature Engineering          #
#                                      #
########################################

def format(Train_Test) :
    keeper = pd.DataFrame()
    for prv in tqdm(fmt_prv) :
        df = pd.read_csv(Train_Test + '/' + prv +'_' + Train_Test + '.csv')
        
        # wind feature
        wd = df['WindDir'] * np.pi / 180
        wv = df['Wind Speed(km/h)']
        df['Wx'] = wv * np.cos(wd)
        df['Wy'] = wv * np.sin(wd)
        df.drop('WindDir',inplace=True,axis=1)

        df.rename(columns= lambda n : prv + '_' +n if n != 'datetime' and n not in [str(e) for e in range(1,13)]  else n , inplace=True)
        
        if len(list(keeper.index)) == 0 :
            keeper = pd.DataFrame(df, columns= df.columns)
        
        else :
            keeper = keeper.set_index('datetime').join(df.set_index('datetime'), on='datetime', how='left', sort=True)
            keeper.reset_index(drop=False,inplace=True)

    # month feature
    for i in range(1,13) :
        keeper[str(i)] = pd.to_datetime(keeper['datetime'], format='%Y-%m-%d %H:%M:%S', errors='ignore').apply(lambda m : int(m.month == i))

    return keeper

train = format('Train')
test = format('Test')

# Data Preview

In [29]:
train

,datetime,chanthaburi_Temp(C),chanthaburi_Wind Speed(km/h),chanthaburi_PM2.5,chanthaburi_Wx,chanthaburi_Wy,chiangmai_Temp(C),chiangmai_Wind Speed(km/h),chiangmai_PM2.5,chiangmai_Wx,chiangmai_Wy,kanchanaburi_Temp(C),kanchanaburi_Wind Speed(km/h),kanchanaburi_PM2.5,kanchanaburi_Wx,kanchanaburi_Wy,bangkok_Temp(C),bangkok_Wind Speed(km/h),bangkok_PM2.5,bangkok_Wx,bangkok_Wy,khonkaen_Temp(C),khonkaen_Wind Speed(km/h),khonkaen_PM2.5,khonkaen_Wx,khonkaen_Wy,songkhla_Temp(C),songkhla_Wind Speed(km/h),songkhla_PM2.5,songkhla_Wx,songkhla_Wy,1,2,3,4,5,6,7,8,9,10,11,12
0,2016-03-03 01:00:00,19.400000,12.000000,27.722282,-11.954336,1.045869,26.200000,14.000000,24.966794,8.030070,11.468129,25.500000,13.000000,25.75206,9.192388,9.192388,24.300000,27.000000,20.672178,6.988114,26.079997,23.100000,4.000000,31.081876,0.694593,3.939231,25.000000,41.000000,17.793265,2.510526e-15,41.000000,0,0,1,0,0,0,0,0,0,0,0,0
1,2016-03-03 02:00:00,19.133333,11.666667,27.722282,-11.646929,-0.678356,25.533333,14.000000,24.966794,8.030070,11.468129,25.333333,12.333333,25.75206,7.927714,9.447881,24.100000,25.666667,20.672178,7.361283,24.588397,22.400000,4.666667,31.081876,2.449891,3.971878,25.000000,40.666667,17.793265,2.490115e-15,40.666667,0,0,1,0,0,0,0,0,0,0,0,0
2,2016-03-03 03:00:00,18.866667,11.333333,27.722282,-11.099194,-2.291799,24.866667,14.000000,24.966794,8.030070,11.468129,25.166667,11.666667,25.75206,6.691725,9.556774,23.900000,24.333333,20.672178,7.653922,23.098238,21.700000,5.333333,31.081876,4.277990,3.184846,25.000000,40.333333,17.793265,2.469704e-15,40.333333,0,0,1,0,0,0,0,0,0,0,0,0
3,2016-03-03 04:00:00,18.600000,11.000000,27.722282,-10.336619,-3.762222,24.200000,14.000000,24.966794,8.030070,11.468129,25.000000,11.000000,25.75206,5.500000,9.526279,23.700000,23.000000,20.672178,7.866463,21.612930,21.000000,6.000000,31.081876,5.795555,1.552914,25.000000,40.000000,17.793265,2.449294e-15,40.000000,0,0,1,0,0,0,0,0,0,0,0,0
4,2016-03-03 05:00:00,19.033333,9.666667,27.722282,-8.876089,-3.828771,24.366667,15.333333,24.966794,7.666667,13.279056,25.466667,12.333333,25.75206,5.853405,10.855817,24.266667,22.000000,20.672178,7.524443,20.673238,20.966667,6.666667,31.081876,5.461014,3.823843,25.133333,39.333333,17.793265,2.287030e+00,39.266788,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26641,2019-03-18 02:00:00,28.000000,10.000000,75.800000,-1.736482,-9.848078,29.900000,10.000000,40.500000,0.871557,9.961947,30.200000,19.000000,42.20000,1.655959,18.927699,27.500000,28.000000,31.900000,11.833311,25.376618,32.000000,5.000000,78.200000,-2.867882,4.095760,27.100000,46.000000,28.200000,2.816688e-15,46.000000,0,0,1,0,0,0,0,0,0,0,0,0
26642,2019-03-18 03:00:00,28.000000,10.000000,75.600000,-1.736482,-9.848078,29.900000,10.000000,43.200000,0.871557,9.961947,30.200000,19.000000,41.20000,1.655959,18.927699,27.500000,28.000000,30.200000,11.833311,25.376618,32.000000,5.000000,80.000000,-2.867882,4.095760,27.100000,46.000000,30.200000,2.816688e-15,46.000000,0,0,1,0,0,0,0,0,0,0,0,0
26643,2019-03-18 04:00:00,28.000000,10.000000,84.600000,-1.736482,-9.848078,29.900000,10.000000,46.900000,0.871557,9.961947,30.200000,19.000000,37.70000,1.655959,18.927699,27.500000,28.000000,32.800000,11.833311,25.376618,32.000000,5.000000,85.500000,-2.867882,4.095760,27.100000,46.000000,30.700000,2.816688e-15,46.000000,0,0,1,0,0,0,0,0,0,0,0,0
26644,2019-03-18 05:00:00,28.000000,10.000000,104.800000,-1.736482,-9.848078,29.900000,10.000000,46.300000,0.871557,9.961947,30.200000,19.000000,39.00000,1.655959,18.927699,27.500000,28.000000,32.000000,11.833311,25.376618,32.000000,5.000000,88.300000,-2.867882,4.095760,27.100000,46.000000,31.500000,2.816688e-15,46.000000,0,0,1,0,0,0,0,0,0,0,0,0


In [30]:
test

,datetime,chanthaburi_Temp(C),chanthaburi_Wind Speed(km/h),chanthaburi_PM2.5,chanthaburi_Wx,chanthaburi_Wy,chiangmai_Temp(C),chiangmai_Wind Speed(km/h),chiangmai_PM2.5,chiangmai_Wx,chiangmai_Wy,kanchanaburi_Temp(C),kanchanaburi_Wind Speed(km/h),kanchanaburi_PM2.5,kanchanaburi_Wx,kanchanaburi_Wy,bangkok_Temp(C),bangkok_Wind Speed(km/h),bangkok_PM2.5,bangkok_Wx,bangkok_Wy,khonkaen_Temp(C),khonkaen_Wind Speed(km/h),khonkaen_PM2.5,khonkaen_Wx,khonkaen_Wy,songkhla_Temp(C),songkhla_Wind Speed(km/h),songkhla_PM2.5,songkhla_Wx,songkhla_Wy,1,2,3,4,5,6,7,8,9,10,11,12
0,2019-03-18 01:00:00,25.600000,13.000000,33.865667,-4.446262,-12.216004,28.400000,12.000000,28.229168,-2.083778,1.181769e+01,29.200000,16.000000,24.789817,-2.778371,15.756924,26.600000,25.000000,19.741368,2.178894e+00,24.904867,29.900000,9.000000,30.132799,-9.000000,1.102182e-15,26.500000,47.000000,17.378282,-8.161464e+00,46.285964,0,0,1,0,0,0,0,0,0,0,0,0
1,2019-03-18 02:00:00,25.000000,13.333333,33.865667,-3.824043,-12.773193,28.100000,12.333333,28.229168,-2.844262,1.200089e+01,29.033333,16.000000,24.789817,-4.141105,15.454813,26.400000,24.666667,19.741368,1.510398e-15,24.666667,29.433333,10.000000,30.132799,-9.961947,8.715574e-01,26.300000,46.666667,17.378282,-6.763487e+00,46.173943,0,0,1,0,0,0,0,0,0,0,0,0
2,2019-03-18 03:00:00,24.400000,13.666667,33.865667,-3.151750,-13.298280,27.800000,12.666667,28.229168,-3.632841,1.213453e+01,28.866667,16.000000,24.789817,-5.472322,15.035082,26.200000,24.333333,19.741368,-2.120790e+00,24.240738,28.966667,11.000000,30.132799,-10.832885,1.910130e+00,26.100000,46.333333,17.378282,-5.378972e+00,46.020044,0,0,1,0,0,0,0,0,0,0,0,0
3,2019-03-18 04:00:00,23.800000,14.000000,33.865667,-2.431074,-13.787309,27.500000,13.000000,28.229168,-4.446262,1.221600e+01,28.700000,16.000000,24.789817,-6.761892,14.500925,26.000000,24.000000,19.741368,-4.167556e+00,23.635386,28.500000,12.000000,30.132799,-11.591110,3.105829e+00,25.900000,46.000000,17.378282,-4.009164e+00,45.824956,0,0,1,0,0,0,0,0,0,0,0,0
4,2019-03-18 05:00:00,24.066667,11.666667,33.865667,-1.354417,-11.587781,27.600000,14.666667,28.229168,-5.809170,1.346717e+01,28.900000,16.333333,24.789817,-8.166667,14.145082,26.333333,21.666667,19.741368,-3.140190e+00,21.437902,28.333333,17.333333,30.132799,-16.605152,4.971256e+00,25.966667,44.666667,17.378282,-2.597136e+00,44.591098,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8801,2020-03-18 18:00:00,32.700000,24.333333,42.900000,-8.322490,-22.865854,33.500000,12.333333,28.400000,-12.249940,1.431813e+00,31.266667,6.333333,24.900000,-2.842395,-5.659673,27.866667,11.333333,22.900000,1.315720e+00,11.256701,32.833333,17.666667,42.700000,-8.833333,-1.529978e+01,27.533333,29.666667,11.700000,-5.151563e+00,29.215963,0,0,1,0,0,0,0,0,0,0,0,0
8802,2020-03-18 19:00:00,29.600000,23.000000,30.800000,-7.866463,-21.612930,31.400000,17.000000,31.500000,-17.000000,2.081900e-15,30.400000,6.000000,27.000000,-1.041889,-5.908847,27.100000,13.000000,27.200000,2.257426e+00,12.802501,32.500000,20.000000,46.500000,-10.000000,-1.732051e+01,26.500000,29.000000,10.900000,-5.035797e+00,28.559425,0,0,1,0,0,0,0,0,0,0,0,0
8803,2020-03-18 20:00:00,28.200000,27.666667,31.500000,-7.160660,-26.723948,30.466667,16.333333,24.100000,-14.997530,-6.469303e+00,29.966667,6.666667,26.100000,-4.285251,-5.106963,26.933333,12.000000,28.200000,2.426611e+00,11.752087,31.800000,20.666667,51.200000,-10.333333,-1.789786e+01,26.166667,29.333333,12.000000,-2.556568e+00,29.221711,0,0,1,0,0,0,0,0,0,0,0,0
8804,2020-03-18 21:00:00,26.800000,32.333333,31.500000,-5.614624,-31.842117,29.533333,15.666667,24.100000,-10.751119,-1.139552e+01,29.533333,7.333333,26.100000,-6.891079,-2.508148,26.766667,11.000000,28.200000,2.536775e+00,10.703494,31.100000,21.333333,51.200000,-10.666667,-1.847521e+01,25.833333,29.666667,12.000000,1.816559e-15,29.666667,0,0,1,0,0,0,0,0,0,0

# Normalize & XY split

In [153]:
def normalize(data):
    temp_data = data.copy()
    mean = temp_data.iloc[:,1:temp_data.shape[1]-12].mean()
    train_mean = mean
    std = temp_data.iloc[:,1:temp_data.shape[1]-12].std()
    train_std = std
    temp_data.iloc[:,1:temp_data.shape[1]-12] = (temp_data.iloc[:,1:temp_data.shape[1]-12] - mean) / std
    return temp_data

normalized_train = normalize(train)
normalized_test = normalize(test)

In [154]:
# train
x_train = normalized_train.drop(['datetime','chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5'],axis=1)
y_train = normalized_train[['chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5']]


print('Before Split')
print('x_train',x_train.shape)
print('y_train',y_train.shape)

from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=69)

print('Split')
print('x_train',x_train.shape)
print('y_train',y_train.shape)
print('x_val',x_val.shape)
print('y_val',y_val.shape)

add_y = pd.DataFrame([list(y_train.mean())] * 72, columns= ['chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5'] ) 
y_train = pd.concat([y_train,add_y])
y_train.reset_index(inplace=True,drop=True)

add_y = pd.DataFrame([list(y_val.mean())] * 72, columns= ['chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5'] ) 
y_val = pd.concat([y_val,add_y])
y_val.reset_index(inplace=True,drop=True)



# test
x_test = normalized_test.drop(['datetime','chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5'],axis=1)
y_test = normalized_test[['chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5']]
temp_label = test[['chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5']]

print('Test')
print('x_test',x_test.shape)
print('y_test',y_test.shape)

add_y = pd.DataFrame([list(y_test.mean())] * 72, columns= ['chanthaburi_PM2.5','chiangmai_PM2.5','kanchanaburi_PM2.5','bangkok_PM2.5','khonkaen_PM2.5','songkhla_PM2.5'] ) 
y_test = pd.concat([y_test,add_y])
y_test.reset_index(inplace=True,drop=True)

print('After add_y')
print('y_train',y_train.shape)
print('y_val',y_val.shape)
print('y_test',y_test.shape)

Before Split
x_train (26646, 36)
y_train (26646, 6)
Split
x_train (21316, 36)
y_train (21316, 6)
x_val (5330, 36)
y_val (5330, 6)
Test
x_test (8806, 36)
y_test (8806, 6)
After add_y
y_train (21388, 6)
y_val (5402, 6)
y_test (8878, 6)


# Model

In [155]:
!mkdir saved_model
!mkdir Graph

mkdir: cannot create directory ‘saved_model’: File exists
mkdir: cannot create directory ‘Graph’: File exists


In [200]:
# hyperparameter 
batch_size = 512
epochs = 100

# set up callback_list
weight_path_model = "saved_model/model_V1.h5"
tensorboard_path = 'Graph/model_V1'
callbacks_list_model = [
    ModelCheckpoint(
            weight_path_model,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=2
        ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=2, min_lr=1e-9)
]

def get_model() :
    input1 = Input(shape=(36,1,))
    x = LSTM(720,kernel_initializer='Orthogonal')(input1)
    x = Dense(512,activation='relu')(x)
    x = Dense(600,activation='relu')(x)
    out = Dense(72, activation='linear')(x)

    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                  loss='mse',
                  metrics=['mse','acc'])
    return model



In [201]:
K.clear_session()

all_pred = []

for i in range(6) :

    weight_path_model = "saved_model/model_"+ fmt_prv[i] +"_V1.h5"
    tensorboard_path = "Graph/model_" + fmt_prv[i] + "_V1"
    callbacks_list_model = [
    TensorBoard(log_dir=tensorboard_path),
    ModelCheckpoint(
            weight_path_model,
            save_best_only=True,
            save_weights_only=True,
            monitor='val_loss',
            mode='min',
            verbose=2
        ),
        ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=2, min_lr=1e-10)
    ]

    # make y -> array data_row x 72
    y = []
    for j in range(x_train.shape[0]) :
        y.append(list(y_train.iloc[j:j+72,i]))
    y = np.array(y)

    real_y_val = []
    for j in range(x_val.shape[0]) :
        real_y_val.append(list(y_val.iloc[j:j+72,i]))
    real_y_val = np.array(real_y_val)

    label = []
    for j in range(temp_label.shape[0]) :
        label.append(list(temp_label.iloc[j*6:j*6+72,i]))
    while len(label) < 1454 :
        label.append(list(temp_label.iloc[temp_label.shape[0]-1:temp_label.shape[0]-1+72,i]))
    label = np.array(label)

    model = get_model()
    model.summary()
    # train
    print(x_train.shape)
    print(y.shape)
    model.fit(x_train, y, epochs=epochs , callbacks=callbacks_list_model,batch_size=batch_size,verbose=1,validation_data=(x_val,real_y_val))

    # predict
    each_pred = []
    for j in range(1454) :
        step_pred = model.predict(np.array(x_test.iloc[j*6,:]).reshape(-1,36))
        each_pred += list(step_pred[0])
    each_pred = (np.array(each_pred) * train[fmt_prv[i]+'_PM2.5'].std()) + train[fmt_prv[i]+'_PM2.5'].mean()
    each_pred = list(each_pred)

    # check rmse
    label_vec = list(itertools.chain.from_iterable(list(label)))
    print('label shape',len(label_vec))
    print('pred shape',len(each_pred))
    
    label_vec
    rms = sqrt(mean_squared_error(label_vec[:100000], each_pred[:100000]))
    print('RMSE',rms)
    all_pred += each_pred



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 36, 1)]           0         
_________________________________________________________________
lstm (LSTM)                  (None, 720)               2079360   
_________________________________________________________________
dense (Dense)                (None, 512)               369152    
_________________________________________________________________
dense_1 (Dense)              (None, 600)               307800    
_________________________________________________________________
dense_2 (Dense)              (None, 72)                43272     
Total params: 2,799,584
Trainable params: 2,799,584
Non-trainable params: 0
_________________________________________________________________
(21316, 36)
(21316, 72)
Epoch 1/100
42/42 [==============================] - 6s 110ms/step - loss: 0.9937 - mse: 0.99

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 36, 1)]           0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 720)               2079360   
_________________________________________________________________
dense_3 (Dense)              (None, 512)               369152    
_________________________________________________________________
dense_4 (Dense)              (None, 600)               307800    
_________________________________________________________________
dense_5 (Dense)              (None, 72)                43272     
Total params: 2,799,584
Trainable params: 2,799,584
Non-trainable params: 0
_________________________________________________________________
(21316, 36)
(21316, 72)
Epoch 1/100
42/42 [==============================] - 6s 109ms/step - loss: 0.9894 - mse: 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


...

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 36, 1)]           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 720)               2079360   
_________________________________________________________________
dense_9 (Dense)              (None, 512)               369152    
_________________________________________________________________
dense_10 (Dense)             (None, 600)               307800    
_________________________________________________________________
dense_11 (Dense)             (None, 72)                43272     
Total params: 2,799,584
Trainable params: 2,799,584
Non-trainable params: 0
_________________________________________________________________
(21316, 36)
(21316, 72)
Epoch 1/100
42/42 [==============================] - 6s 117ms/step - loss: 0.9977 - mse: 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 36, 1)]           0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 720)               2079360   
_________________________________________________________________
dense_12 (Dense)             (None, 512)               369152    
_________________________________________________________________
dense_13 (Dense)             (None, 600)               307800    
_________________________________________________________________
dense_14 (Dense)             (None, 72)                43272     
Total params: 2,799,584
Trainable params: 2,799,584
Non-trainable params: 0
_________________________________________________________________
(21316, 36)
(21316, 72)
Epoch 1/100
42/42 [==============================] - 6s 109ms/step - loss: 0.9911 - mse: 0.

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 36, 1)]           0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 720)               2079360   
_________________________________________________________________
dense_15 (Dense)             (None, 512)               369152    
_________________________________________________________________
dense_16 (Dense)             (None, 600)               307800    
_________________________________________________________________
dense_17 (Dense)             (None, 72)                43272     
Total params: 2,799,584
Trainable params: 2,799,584
Non-trainable params: 0
_________________________________________________________________
(21316, 36)
(21316, 72)
Epoch 1/100
42/42 [==============================] - 6s 110ms/step - loss: 1.0022 - mse: 1.

In [202]:
out_df = pd.DataFrame(np.array([[ i for i in range(len(all_pred))],all_pred]).T,columns=['Id','Expected'])
out_df['Id'] =  out_df['Id'].apply(lambda x : int(x+1))
out_df

,Id,Expected
0,1,64.821426
1,2,27.644966
2,3,30.120663
3,4,29.292574
4,5,30.180630
...,...,...
628123,628124,18.060726
628124,628125,17.780706
628125,628126,18.008457
628126,628127,17.878519


In [203]:
out_df.to_csv('submission.csv',index=False)